In [ ]:
import firedrake

nx, ny = 64, 64
mesh = firedrake.UnitSquareMesh(nx, ny, diagonal="crossed")
Q = firedrake.FunctionSpace(mesh, "Bernstein", 2)

In [ ]:
from firedrake import max_value, sqrt, inner, as_vector, Constant

def make_obstacle(mesh):
    x = firedrake.SpatialCoordinate(mesh)
    y = as_vector((1/2, 1/2))
    z = 1/4
    return sqrt(max_value(z**2 - inner(x - y, x - y), 0))

g = firedrake.project(make_obstacle(mesh), Q)

In [ ]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
fig = plt.figure()
axes = fig.add_subplot(projection='3d')
firedrake.trisurf(g, axes=axes);

In [ ]:
from firedrake import grad, dx

u = firedrake.Function(Q)
J = 0.5 * inner(grad(u), grad(u)) * dx

In [ ]:
bcs = firedrake.DirichletBC(Q, 0, "on_boundary")
F = firedrake.derivative(J, u)
problem = firedrake.NonlinearVariationalProblem(F, u, bcs)
params = {
    "solver_parameters": {
        "snes_type": "vinewtonrsls",
        "ksp_type": "gmres",
        "pc_type": "lu",
    }
}
solver = firedrake.NonlinearVariationalSolver(problem, **params)

In [ ]:
upper = firedrake.Function(Q)
upper.assign(Constant(10e3));

In [ ]:
solver.solve(bounds=(g, upper))

In [ ]:
fig = plt.figure()
axes = fig.add_subplot(projection='3d')
firedrake.trisurf(u, axes=axes);

In [ ]:
δu = firedrake.project(u - g, Q)

fig, axes = plt.subplots()
axes.set_aspect("equal")
colors = firedrake.tripcolor(δu, axes=axes)
fig.colorbar(colors);